# Visualize TimePix3 data

read from "hdf5" files created by Kuepper et al. (Hubertus Bromberger)

### Imports

In [19]:
# %matplotlib widget
%pylab qt   

Populating the interactive namespace from numpy and matplotlib


In [20]:
import numpy as np  
import matplotlib.pyplot as plt
import h5py
import os.path

### Define HDF path & file

In [21]:
hdf_file_path = '/home/cpassow/jupyterlab/CAMP/data/'
hdf_file_name = '20191031-052753_CH3I_000000.hdf5'
hdf_file_complete_path = hdf_file_path + hdf_file_name

assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'

### Utility function

In [111]:
def data_from_hdf(hdf_file_complete_path, event_type = 'centroided'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
    print('Reading from HDF5 - number of events: {:.2e}'.format(len(tof)))
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'centroided'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
    print('Slicing - number of events: {:.2e} | {:.2%}'.format(len(sliced_tof), len(sliced_tof)/len(tof)))
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, hist_bins=100, time_unit = None):     
    '''Plot time-of-flight spectrum via histogram'''
    fig = plt.subplots(num = 1)
    plt.clf()
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    plt.hist(tof, bins = hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
        
def plot_1d_histograms(tof, x_pos, y_pos, time_unit = None):      
    '''1D plots - detector position vs time-of-flight for each dimension'''
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    fig = plt.figure(num = 2)
    plt.clf()
    plt.plot(tof, x_pos,'.')
    plt.title("ToF vs x_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos [px]')
    
    fig = plt.figure(num=3)
    plt.clf()
    plt.plot(tof, y_pos,'.')
    plt.title("ToF vs y_pos")
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos [px]')
    plt.show()

def plot_2d_histograms(tof, x_pos, y_pos ,bin_tof = 2000, bin_space = 200, time_unit = None):
    '''2D plot (heatmap) - detector position vs time-of-flight for each dimension'''
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    fig = plt.figure(num = 4)
    plt.clf()
    plt.hist2d(tof, x_pos, bins= (bin_tof,bin_space))
    plt.title('2d histogram: time-of-flight / x_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('x_pos')

    fig = plt.figure(num = 5)
    plt.clf()
    plt.hist2d(tof, y_pos,bins= (bin_tof,bin_space))
    plt.title('2d histogram: time-of-flight / y_pos')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('y_pos')
    plt.show()
    
def vmi_image(x_pos, y_pos):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos,bins= (10, 10), cmax= 1000)
    plt.colorbar()
    plt.show()
    return counts, xbins, ybins, image
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'centroided'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
# def transform_vmi_to_polar(x_pos, y_pos):
#     image = vmi_image(x_pos, y_pos)
#     print(len)

    
    
    
    
    
####
    
tof_start = 6.5E-6
tof_end = 7.7E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)

counts, xbins, ybins, image =  vmi_image(x_pos, y_pos)


tt = np.flipud(counts.transpose())
    


fig = plt.figure(num = 7)
plt.clf()
plt.title('transpose')
image1 = plt.imshow(tt,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()])





Reading from HDF5 - number of events: 3.76e+05
Slicing - number of events: 1.08e+04 | 2.88%


In [47]:
tof_start = 6.5E-6
tof_end = 7.7E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)

transform_vmi_to_polar(x_pos, y_pos)

Reading from HDF5 - number of events: 3.76e+05
Slicing - number of events: 1.08e+04 | 2.88%
<class 'tuple'>
4


### Show complete time-of-flight

In [23]:
tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path)

plot_tof(tof, hist_bins = 100)

Reading from HDF5 - number of events: 3.76e+05


### Reduce data to one single FEL train

In [24]:
tof_start = 0E-6
tof_end = 12E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)

Reading from HDF5 - number of events: 3.76e+05
Slicing - number of events: 3.58e+05 | 95.27%


### Show time-of-flight spectrum

In [25]:
plot_tof(tof, hist_bins = 1000, time_unit = 'micro')

### Show X,Y position vs time-of-flight (1d)

In [26]:
plot_1d_histograms(tof, x_pos, y_pos, time_unit = 'micro')

### Show X,Y position vs time-of-flight (2d)

In [27]:
plot_2d_histograms(tof, x_pos, y_pos, time_unit = 'micro')

### Show VMI for time-of-flight interval

In [35]:
tof_start = 0E-6
tof_end = 12E-6

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)
plot_tof(tof, hist_bins = 1000, time_unit = 'micro')

Reading from HDF5 - number of events: 3.76e+05
Slicing - number of events: 3.58e+05 | 95.27%


In [36]:
tof_start = 6.5E-6
tof_end = 7.7E-6

display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start, tof_end, time_unit = 'micro')

Reading from HDF5 - number of events: 3.76e+05
Slicing - number of events: 1.08e+04 | 2.88%
